# CSC421 Assignment 1 Search #

This notebook is based on the supporting material for topics covered in **Chapter 3 - Intelligent Agents** from the book *Artificial Intelligence: A Modern Approach.* You can consult and modify the code provided in search.py and search.ipynb for completing the assignment questions. 

## Introduction - Block World 

In this assignment we will explore search algorithm using a Block World consisting of a width by height grid of 
cells. That way you can reuse the ipythonblocks code from agents.ipynb and agents.py for visualization. A Block World navigation problem consists of finding a path from a starting location on the grid to a destination location of the grid. In each square/cell the search agent can move in 4 directions (UP, LEFT, RIGHT, DOWN) except the edges in which one of these directions is not available depending on which side of the grid the agent is on. 

In the simplest version of this problem the search agent just has to go from the starting cell to the destination cell and a solution is the sequence of actions required. Note that for this particular problem search algoithms are overkill and instead a very simple algorithm can be used to find immediately a solution. One can take the difference of the x-coordinates and then move the appropriate number of LEFT or RIGHT steps and similarly take the difference of the y-coodinates and then move the appropriate number of UP and DOWN steps. However implementing search algorithms for this problem will help you understand the concepts better and is also needed 
for the more complex version of the problem which includes obstacles. 

In the obstacle version of the problem you are also provided with the location of a number of squares (expressed as (x,y) coordinates that act as obstacles. For example if there is an obstacle at a particular location then the agent will not be able to move to that location. Now a solution is a sequence of actions that takes our agents from the starting location to the destination location that also avoids any obstacles. 

In the rest of the notebook we specify the different subproblems based on this setting that you will be working on for this assignment. 

# Question 2A (Minimum) 1 point

Write a subclass of the abstract class problem provided in search.ipynb. Similarly to how in search.ipynb the abstract problem is made concrete by GraphProblem in the case of searching a graph, you will make it concrete by writing BlockWorldProblem. 



In [1]:
# YOUR CODE GOES HERE 

from agents import *
from search import *
from notebook import psource, heatmap, gaussian_kernel, show_map, final_path_colors, display_visual, plot_NQueens
import math

class BlockWorldProblem(Problem):

    def __init__(self, initial, goal, width, height):
        super().__init__(initial, goal)
        self.width = width
        self.height = height

    def actions(self, state):
        # state will be a tuple of (x,y) location from upper left corner
        possible_actions = ['UP','DOWN','LEFT','RIGHT']
        if state[0] == 0:
            possible_actions.remove('LEFT')
        if state[0] == (self.width-1):
            possible_actions.remove('RIGHT')
        if state[1] == 0:
            possible_actions.remove('UP')
        if state[1] == (self.height-1):
            possible_actions.remove('DOWN')
        return possible_actions

    def result(self, state, action):
        # resulting position after taking the specified action
        new_state = list(state)
        if action == 'UP':
            new_state[1] -= 1
        if action == 'DOWN':
            new_state[1] += 1
        if action == 'LEFT':
            new_state[0] -= 1
        if action == 'RIGHT':
            new_state[0] += 1
        return tuple(new_state)

    def goal_test(self, state):
        # Given a state, return True if state is a goal state or False, otherwise
        return state == self.goal
    
    def h(self,node):
        # linear distance heuristic
        current_state = list(node.state)
        goal_state = list(self.goal)
        return math.sqrt( (goal_state[0]-current_state[0])**2 + (goal_state[1]-current_state[1])**2)
    


# Question 2B (Minimum) 1 point 

In the search.ipynb notebook there is visualization code for Breadth-First Graph search and Depth-First Graph search. Remove the graph visualization code but keep the search code the same. Change the code so that the frontier gets printed as the search algorithm iterates. For debugging purposes you can start with the Romania example first but in the final solution you should show how the frontier changes for a particular instances of the BlockWorld problem. Use the following dimensions: a 10 by 10 grid with a starting location of (0,1) and a destination location of (7,8). Notice that in our BlockWorld problem there are a lot of overlapping path reaching a node so it makes sense to use GraphSearch and keep track of what nodes have been expanded already so that they are not expanded again. 


In [2]:
# YOUR CODE GOES HERE 

def breadth_first_graph_search(problem):
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    
    frontier = deque([node])
    
    explored = set()
    
    while frontier:
        
        print("Frontier: " + str([f.state for f in frontier]))  # print frontier
        
        node = frontier.popleft()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    return child
                frontier.append(child)
    return None

def depth_first_graph_search(problem):

    frontier = [(Node(problem.initial))]

    explored = set()
    while frontier:
        print("Frontier: "+str([f.state for f in frontier]))  # print frontier
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and child not in frontier)
    return None

# Test with 10 by 10 board with start (0,1) and goal (7,8)
block_problem = BlockWorldProblem((0,1),(7,8),10,10)
print("Breadth first:")
node = breadth_first_graph_search(block_problem)

print("\nDepth first:")
node = depth_first_graph_search(block_problem)

Breadth first:
Frontier: [(0, 1)]
Frontier: [(0, 0), (0, 2), (1, 1)]
Frontier: [(0, 2), (1, 1), (1, 0)]
Frontier: [(1, 1), (1, 0), (0, 3), (1, 2)]
Frontier: [(1, 0), (0, 3), (1, 2), (2, 1)]
Frontier: [(0, 3), (1, 2), (2, 1), (2, 0)]
Frontier: [(1, 2), (2, 1), (2, 0), (0, 4), (1, 3)]
Frontier: [(2, 1), (2, 0), (0, 4), (1, 3), (2, 2)]
Frontier: [(2, 0), (0, 4), (1, 3), (2, 2), (3, 1)]
Frontier: [(0, 4), (1, 3), (2, 2), (3, 1), (3, 0)]
Frontier: [(1, 3), (2, 2), (3, 1), (3, 0), (0, 5), (1, 4)]
Frontier: [(2, 2), (3, 1), (3, 0), (0, 5), (1, 4), (2, 3)]
Frontier: [(3, 1), (3, 0), (0, 5), (1, 4), (2, 3), (3, 2)]
Frontier: [(3, 0), (0, 5), (1, 4), (2, 3), (3, 2), (4, 1)]
Frontier: [(0, 5), (1, 4), (2, 3), (3, 2), (4, 1), (4, 0)]
Frontier: [(1, 4), (2, 3), (3, 2), (4, 1), (4, 0), (0, 6), (1, 5)]
Frontier: [(2, 3), (3, 2), (4, 1), (4, 0), (0, 6), (1, 5), (2, 4)]
Frontier: [(3, 2), (4, 1), (4, 0), (0, 6), (1, 5), (2, 4), (3, 3)]
Frontier: [(4, 1), (4, 0), (0, 6), (1, 5), (2, 4), (3, 3), (4, 2)]


In [3]:
# DELETE ME -- testing code
romania_map = UndirectedGraph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

romania_map.locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))

romania_problem = GraphProblem('Arad', 'Bucharest', romania_map)

print("Breadth first:")
node = breadth_first_graph_search(romania_problem)

print("\nDepth first:")
# run depth first search
node = depth_first_graph_search(romania_problem)

Breadth first:
Frontier: ['Arad']
Frontier: ['Zerind', 'Sibiu', 'Timisoara']
Frontier: ['Sibiu', 'Timisoara', 'Oradea']
Frontier: ['Timisoara', 'Oradea', 'Fagaras', 'Rimnicu']
Frontier: ['Oradea', 'Fagaras', 'Rimnicu', 'Lugoj']
Frontier: ['Fagaras', 'Rimnicu', 'Lugoj']

Depth first:
Frontier: ['Arad']
Frontier: ['Zerind', 'Sibiu', 'Timisoara']
Frontier: ['Zerind', 'Sibiu', 'Lugoj']
Frontier: ['Zerind', 'Sibiu', 'Mehadia']
Frontier: ['Zerind', 'Sibiu', 'Drobeta']
Frontier: ['Zerind', 'Sibiu', 'Craiova']
Frontier: ['Zerind', 'Sibiu', 'Rimnicu', 'Pitesti']
Frontier: ['Zerind', 'Sibiu', 'Rimnicu', 'Bucharest']


# Question 2C (Minimum) 1 point 

Using the code you wrote for the previous question to print the frontier as the search algorithm iterates show how the frontier changes for Greedy and A* graph search. Use the straight-line heuristic. 

In [4]:
# YOUR CODE GOES HERE

def best_first_graph_search(problem, f):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    
    explored = set()
    while frontier:
        # print frontier by popping all elements to get the states and appending all afterwords
        popped_frontier = PriorityQueue('min', f)
        frontier_list = []
        for i in range(0,len(frontier)):
            temp_node = frontier.pop()
            frontier_list.append(temp_node.state)
            popped_frontier.append(temp_node)
        for i in range(0,len(popped_frontier)):
            frontier.append(popped_frontier.pop())
        print("Frontier: "+ str(frontier_list))
        
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None

def greedy_best_first_search(problem, h=None):
    """Greedy Best-first graph search is an informative searching algorithm with f(n) = h(n).
    You need to specify the h function when you call best_first_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    node = best_first_graph_search(problem, lambda n: h(n))
    return(node)

def astar_search_graph(problem, h=None):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    node = best_first_graph_search(problem,lambda n: n.path_cost + h(n))
    return(node)

# Test with 10 by 10 board with start (0,1) and goal (7,8)
block_problem = BlockWorldProblem((0,1),(7,8),10,10)

print("Greedy search:")
node = greedy_best_first_search(block_problem)

print("\nA* search:")
node = astar_search_graph(block_problem)

Greedy search:
Frontier: [(0, 1)]
Frontier: [(0, 2), (1, 1), (0, 0)]
Frontier: [(1, 2), (0, 3), (1, 1), (0, 0)]
Frontier: [(1, 3), (2, 2), (0, 3), (1, 1), (0, 0)]
Frontier: [(2, 3), (1, 4), (2, 2), (0, 3), (1, 1), (0, 0)]
Frontier: [(2, 4), (3, 3), (1, 4), (2, 2), (0, 3), (1, 1), (0, 0)]
Frontier: [(3, 4), (2, 5), (3, 3), (1, 4), (2, 2), (0, 3), (1, 1), (0, 0)]
Frontier: [(3, 5), (4, 4), (2, 5), (3, 3), (1, 4), (2, 2), (0, 3), (1, 1), (0, 0)]
Frontier: [(4, 5), (3, 6), (4, 4), (2, 5), (3, 3), (1, 4), (2, 2), (0, 3), (1, 1), (0, 0)]
Frontier: [(4, 6), (5, 5), (3, 6), (4, 4), (2, 5), (3, 3), (1, 4), (2, 2), (0, 3), (1, 1), (0, 0)]
Frontier: [(5, 6), (4, 7), (5, 5), (3, 6), (4, 4), (2, 5), (3, 3), (1, 4), (2, 2), (0, 3), (1, 1), (0, 0)]
Frontier: [(5, 7), (6, 6), (4, 7), (5, 5), (3, 6), (4, 4), (2, 5), (3, 3), (1, 4), (2, 2), (0, 3), (1, 1), (0, 0)]
Frontier: [(6, 7), (5, 8), (6, 6), (4, 7), (5, 5), (3, 6), (4, 4), (2, 5), (3, 3), (1, 4), (2, 2), (0, 3), (1, 1), (0, 0)]
Frontier: [(6, 8),

# Question 2D (Expected) 1 point 

Modify the problem definition to take into account a list of random obstacles expressed as (x,y) coordinates. 
Write code to generate random instances of such problems. To generate a random instances of a BlockWorld problem 
with width W and height H use the following algorith: 

1. Generate two random numbers between 0 and W-1 and 0 and H-1 to use as the starting location 
2. Similarly generate two random numbers to use as the starting location 
3. Similarly generate the coordinates of obstacles 

The number of obstacles should be specified as a density parameter which is the percentage of the total 
number of cells that have obstacles. For example in a 6 by 6 grid there are 36 blocks. A density of 0.33 
would mean that there are 12 randomly placed obstacles. 

Show with print statements that your code is working as expected. 
Use ipythonblocks (you can check agents.py and agents.ipynb for examples of usage) to display the 
randomly generated instances of the problem. 

In [8]:
# YOUR CODE GOES HERE 

from math import floor, ceil

class BlockWorldProblem(Problem):
    """The problem of searching a graph from one node to another."""
        
    def __init__(self, width, height, density, visualize=False):
        self.width = width
        self.height = height
        self.grid = BlockGrid(width, height, fill=(200, 200, 200))
        initial = tuple([random.randint(0,width-1),random.randint(0,height-1)])
        goal = tuple([random.randint(0,width-1),random.randint(0,height-1)])
        super().__init__(initial, goal)
        self.density = density
        
        # generate obstacles, making sure they are not the start or end position
        obstacles = []
        while len(obstacles) < width*height*density:
            new_obstacle = tuple([random.randint(0,width-1),random.randint(0,height-1)])
            if new_obstacle != initial and new_obstacle != goal and new_obstacle not in obstacles:
                obstacles.append(new_obstacle)
        self.obstacles = obstacles
        
        # plot the generated board
        self.grid[self.initial[1],self.initial[0]] = (0,255,0)
        self.grid[self.goal[1],self.goal[0]] = (255,0,0)
        for obs in self.obstacles:
            self.grid[obs[1],obs[0]] = (0,0,0)
        
        if visualize:  # print initial, goal, obstacle coordinates and show the board
            print("Initial: " + str(initial))
            print("Goal: " + str(goal))
            print("Obstacles: " + str(obstacles))
            self.grid.show()

    def actions(self, state):
        # state will be a tuple of [x,y] location from upper left corner
        possible_actions = ['UP','DOWN','LEFT','RIGHT']
        if state[1] == 0 or self.result(state,'UP') in self.obstacles:
            possible_actions.remove('UP')
        if state[1] == (self.height-1) or self.result(state,'DOWN') in self.obstacles:
            possible_actions.remove('DOWN')
        if state[0] == 0 or self.result(state,'LEFT') in self.obstacles:
            possible_actions.remove('LEFT')
        if state[0] == (self.width-1)  or self.result(state,'RIGHT') in self.obstacles:
            possible_actions.remove('RIGHT')
        return possible_actions

    def result(self, state, action):
        new_state = list(state)
        if action == 'UP':
            new_state[1] -= 1
        if action == 'DOWN':
            new_state[1] += 1
        if action == 'LEFT':
            new_state[0] -= 1
        if action == 'RIGHT':
            new_state[0] += 1
        return tuple(new_state)

    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """
        return state == self.goal
    
    def h(self,node):
        current_state = list(node.state)
        goal_state = list(self.goal)
        return math.sqrt( (goal_state[0]-current_state[0])**2 + (goal_state[1]-current_state[1])**2)

block_problem = BlockWorldProblem(10,10,.3,True)

Initial: (9, 1)
Goal: (1, 8)
Obstacles: [(4, 7), (2, 2), (8, 0), (3, 5), (8, 9), (3, 9), (3, 0), (1, 5), (7, 7), (2, 9), (5, 3), (7, 3), (0, 5), (8, 7), (9, 5), (2, 7), (6, 3), (7, 6), (9, 9), (9, 3), (6, 5), (4, 9), (9, 7), (4, 8), (9, 2), (3, 1), (1, 2), (7, 5), (6, 6), (5, 1)]


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


# Question 2E (Expected) 1 point

Visualize search algorithms using ipythonblocks with the same color conventions that are used in search.ipynb for visualizing search with the Romania map. Your code should update the color after every iteration similar to how the visualization works in search.ipynb. Show how DFS and A* graph search will look like using this visualization for a 10 by 10 grid with 0.1 density. 

In [12]:
# YOUR CODE GOES HERE 

# orange (255,165,0) frontier
# green (0,255,0) final solution
# red (255,0,0) currently exploring
# white (255,255,255) unexplored
# grey (200,200,200) explored

import time

def graph_search_for_vis(problem):
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    If two paths reach a state, only use the first one. [Figure 3.7]"""
    # we use these two variables at the time of visualisations
    iterations = 0
    all_node_colors = []
    node_colors = {}
    
    frontier = [(Node(problem.initial))]
    explored = set()
    
    # modify the color of frontier nodes to orange
    node_colors[Node(problem.initial).state] = (255,165,0)
    iterations += 1
    all_node_colors.append(dict(node_colors))
      
    while frontier:
        
        # Popping first node of stack
        node = frontier.pop()
        
        # modify the currently searching node to red
        node_colors[node.state] = (255,0,0)
        iterations += 1
        all_node_colors.append(dict(node_colors))
        
        if problem.goal_test(node.state):
            # modify goal node to green after reaching the goal
            node_colors[node.state] = (0,255,0)
            iterations += 1
            all_node_colors.append(dict(node_colors))
            return(iterations, all_node_colors, node)
        
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
        
        for n in frontier:
            # modify the color of frontier nodes to orange
            node_colors[n.state] = (255,165,0)
            iterations += 1
            all_node_colors.append(dict(node_colors))

        # modify the color of explored nodes to gray
        node_colors[node.state] = (200,200,200)
        iterations += 1
        all_node_colors.append(dict(node_colors))
        
    return (iterations, all_node_colors, None)

def best_first_graph_search_vis(problem, f):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    
    # we use these two variables at the time of visualisations
    iterations = 0
    all_node_colors = []
    node_colors = {}
    
    
    f = memoize(f, 'f')
    node = Node(problem.initial)
    
    node_colors[node.state] = (255,0,0)#"red"
    iterations += 1
    all_node_colors.append(dict(node_colors))
    
    if problem.goal_test(node.state):
        node_colors[node.state] = (0,255,0)#"green"
        iterations += 1
        all_node_colors.append(dict(node_colors))
        return(iterations, all_node_colors, node)
    
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    
    node_colors[node.state] = (255,165,0)#"orange"
    iterations += 1
    all_node_colors.append(dict(node_colors))
    
    explored = set()
    while frontier:
        # print frontier by popping all elements to get the states and appending all afterwords
        popped_frontier = PriorityQueue('min', f)
        frontier_list = []
        for i in range(0,len(frontier)):
            temp_node = frontier.pop()
            frontier_list.append(temp_node.state)
            popped_frontier.append(temp_node)
        for i in range(0,len(popped_frontier)):
            frontier.append(popped_frontier.pop())
        print("Frontier: "+ str(frontier_list))
        
        node = frontier.pop()
        
        node_colors[node.state] = (255,0,0)#"red"
        iterations += 1
        all_node_colors.append(dict(node_colors))
        
        
        if problem.goal_test(node.state):
            node_colors[node.state] = (0,255,0)#"green"
            iterations += 1
            all_node_colors.append(dict(node_colors))
            return(iterations, all_node_colors, node)
        
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
                node_colors[child.state] = (255,165,0)#"orange"
                iterations += 1
                all_node_colors.append(dict(node_colors))
            elif child in frontier:
                if f(child) < frontier[child]:
                    del ffrontier[child]
                    frontier.append(child)
                    node_colors[child.state] = (255,165,0)#"orange"
                    iterations += 1
                    all_node_colors.append(dict(node_colors))

        node_colors[node.state] = (200,200,200)#"gray"
        iterations += 1
        all_node_colors.append(dict(node_colors))
    print("returning none")
    return None

def astar_search_graph_vis(problem, h=None):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    (iterations, all_node_colors, node) = best_first_graph_search_vis(problem,lambda n: n.path_cost + h(n))
    return(iterations, all_node_colors, node)

    
def visualize_search(problem,algorithm):
    (iterations, all_node_colors, node) = algorithm(problem)
    if node is None:
        print("This particular block world has no solution. Please run again.")
        return
    
    grid = BlockGrid(problem.width, problem.height, fill=(245,245,245))
    grid.show()
    for itr in range(iterations):
        clear_output()
        for k,v in all_node_colors[itr].items():
            grid[k[1],k[0]] = v
        grid.show()
        time.sleep(0.05)
    solution = node.solution()
    temp_node = problem.initial
    clear_output()
    for action in solution:
        grid[temp_node[1],temp_node[0]] = (0,255,0)
        temp_node = problem.result(temp_node,action)
    grid.show()
    
block_problem = BlockWorldProblem(10,10,.1,False)
visualize_search(block_problem, graph_search_for_vis)


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


In [13]:
print("\nA*:")
visualize_search(block_problem,astar_search_graph_vis)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


# Question 2F (Expected) 1 point 

Modify the search code so that it tracks the number of nodes visited (time complexity) and the maximum size in terms of number of nodes in the frontier (space complexity) during the execution of the search algorithm. When running one instance of the problem you should get two integer numbers: the space complexity (in number of nodes) and the time complexity (in number of nodes). Run 100 randomly generated instances of 10 by 10 grids with 0.1 density and report the average measured time and space complexity for the following algorithms: 
BFGS, DFGS, Greedy GS, A* Graph Search. 

In [14]:
# YOUR CODE GOES HERE 

def breadth_first_graph_search_complexity(problem):
    """[Figure 3.11]
    Note that this function can be implemented in a
    single line as below:
    return graph_search(problem, FIFOQueue())
    """
    time_complexity = 0
    space_complexity = 0
    
    node = Node(problem.initial)
    
    if problem.goal_test(node.state):
        time_complexity += 1
        return (node,time_complexity,space_complexity)
    
    frontier = deque([node])
    
    explored = set()
    while frontier:
        if len(frontier) > space_complexity:
            space_complexity = len(frontier)
        
        node = frontier.popleft()
        
        explored.add(node.state)
        time_complexity += 1
        
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    time_complexity += 1
                    return (child,time_complexity,space_complexity)
                frontier.append(child)
                # need to check here as the frontier could grow and function returns before checking frontier size
                if len(frontier) > space_complexity:  
                    space_complexity = len(frontier)
                
    return (None,time_complexity,space_complexity)

def depth_first_graph_search_complexity(problem):
    """
    [Figure 3.7]
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    time_complexity = 0
    space_complexity = 0
    
    frontier = [(Node(problem.initial))]  # Stack

    explored = set()
    while frontier:
        if len(frontier) > space_complexity:
            space_complexity = len(frontier)

        node = frontier.pop()
        if problem.goal_test(node.state):
            time_complexity += 1
            return (node,time_complexity,space_complexity)
        
        explored.add(node.state)
        time_complexity += 1
        
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and child not in frontier)
        
    return (None,time_complexity,space_complexity)

def best_first_graph_search_complexity(problem, f):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    
    time_complexity = 0
    space_complexity = 0
    
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    
    explored = set()
    while frontier:
        if len(frontier) > space_complexity:
            space_complexity = len(frontier)
        
        node = frontier.pop()
        if problem.goal_test(node.state):
            time_complexity += 1
            return (node,time_complexity,space_complexity)
        
        explored.add(node.state)
        time_complexity += 1
        
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
                if len(frontier) > space_complexity:
                    space_complexity = len(frontier)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return (None,time_complexity,space_complexity)

def greedy_best_first_search_complexity(problem, h=None):
    """Greedy Best-first graph search is an informative searching algorithm with f(n) = h(n).
    You need to specify the h function when you call best_first_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    (node,time_complexity,space_complexity) = best_first_graph_search_complexity(problem, lambda n: h(n))
    return(node,time_complexity,space_complexity)

def astar_search_graph_complexity(problem, h=None):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    (node,time_complexity,space_complexity) = best_first_graph_search_complexity(problem,lambda n: n.path_cost + h(n))
    return(node,time_complexity,space_complexity)

# 100 x generate grid, run each alg
# calculate and print average
count = 0
bfgs_time = 0
bfgs_space = 0
dfgs_time = 0
dfgs_space = 0
greedygs_time = 0
greedygs_space = 0
astar_time = 0
astar_space = 0

while count < 100:
#     generate problem
    block_problem = BlockWorldProblem(10,10,.1)
    
    (node,time_complexity,space_complexity) = breadth_first_graph_search_complexity(block_problem)
    if node is None:  #problem is not solvable
        continue
    bfgs_time += time_complexity
    bfgs_space += space_complexity
    
    (node,time_complexity,space_complexity) = depth_first_graph_search_complexity(block_problem)
    dfgs_time += time_complexity
    dfgs_space += space_complexity
    
    (node,time_complexity,space_complexity) = greedy_best_first_search_complexity(block_problem)
    greedygs_time += time_complexity
    greedygs_space += space_complexity
    
    (node,time_complexity,space_complexity) = astar_search_graph_complexity(block_problem)
    astar_time += time_complexity
    astar_space += space_complexity
    count += 1

# print results
print('{:10}\t{:>10}\t{:>10}'.format('','Time','Space'))
print('-------------------------------------------')
print('{:10}\t{:10.2f}\t{:10.2f}'.format('BFSG',(bfgs_time / count),(bfgs_space / count)))
print('{:10}\t{:10.2f}\t{:10.2f}'.format('DFSG',(dfgs_time / count),(dfgs_space / count)))
print('{:10}\t{:10.2f}\t{:10.2f}'.format('Greedy GS',(greedygs_time / count),(greedygs_space / count)))
print('{:10}\t{:10.2f}\t{:10.2f}'.format('A* GS',(astar_time / count),(astar_space / count)))

          	      Time	     Space
-------------------------------------------
BFSG      	     33.42	      9.39
DFSG      	     41.16	     24.97
Greedy GS 	      7.21	      9.63
A* GS     	     14.79	     10.61


# Question 2G (Advanced) 1 point 

Implement one of the maze generation algorithms described below: https://en.wikipedia.org/wiki/Maze_generation_algorithm

Use this technique to convert your generated maze to blockwise geometry 
https://weblog.jamisbuck.org/2015/10/31/mazes-blockwise-geometry.html

Visualize your generated maze using ipythonblocks. 

In [15]:
# YOUR CODE GOES HERE 

class maze_problem(Problem):
    
    def __init__(self,width,height,display_maze=False):
        kruskal_width = math.ceil(width/2)
        kruskal_height = math.ceil(height/2)
        self.width = width
        self.height = height
        walls = self.create_maze_kruskals(kruskal_width, kruskal_height)
        # convert location of walls to blockwise geometry
        obstacles = set()
        for i in range(0,kruskal_width):
            for j in range(0,kruskal_height):
                if [(i,j),(i+1,j)] in walls:
                    obstacles.add(((i*2)+1,j*2))
                if [(i,j),(i,j+1)] in walls:
                    obstacles.add((i*2,(j*2)+1))
                obstacles.add(((i*2)+1,(j*2)+1))
            
        self.obstacles = obstacles
        # initialize the initial and goal as random, but not an obstacle location
        while True:
            initial = tuple([random.randint(0,width-1),random.randint(0,height-1)])
            if initial not in obstacles:
                break
        while True:
            goal = tuple([random.randint(0,width-1),random.randint(0,height-1)])
            if goal not in obstacles:
                break
        super().__init__(initial, goal)
        self.grid = BlockGrid(width,height,fill=(200,200,200))
        for obs in obstacles:
            if obs[1]<height and obs[0]<width:
                self.grid[obs[1],obs[0]] = (0,0,0)
        self.grid[initial[1],initial[0]] = (0,255,0)
        self.grid[goal[1],goal[0]] = (255,0,0)
        if display_maze:
            self.grid.show()
        
    
    def create_maze_kruskals(self,width,height):
        # identify the set that a cell is in using a dict of form "(x,y):set#""
        cell_sets = {}
        n = 0
        for i in range(0,width):
            for j in range(0,height):
                cell_sets[(i,j)] = n
                n += 1

        # model walls as a pair of tuples with the corresponding cell coordinates
        # eg [(0,0),(1,0)] is the wall between cells (0,0) and (1,0)
        walls = []
        for i in range(0,width):
            for j in range(0,height):
                if i < (width-1):
                    walls.append([(i,j),(i+1,j)])
                if j < (height-1):
                    walls.append([(i,j),(i,j+1)])

        visited_walls = []  # walls that remain

        while len(walls) != 0:
            current_wall = random.choice(walls)
            walls.remove(current_wall)

            c0_set = cell_sets[current_wall[0]]
            c1_set = cell_sets[current_wall[1]]

            if c0_set != c1_set:
                c1_keys = [k for k,v in cell_sets.items() if v == c1_set]
                for k in c1_keys:
                    cell_sets[k] = c0_set
            else:
                visited_walls.append(current_wall)
        return visited_walls
    
    def actions(self, state):
        # state will be a tuple of [x,y] location from upper left corner
        possible_actions = ['UP','DOWN','LEFT','RIGHT']
        if state[1] == 0 or self.result(state,'UP') in self.obstacles:
            possible_actions.remove('UP')
        if state[1] == (self.height-1) or self.result(state,'DOWN') in self.obstacles:
            possible_actions.remove('DOWN')
        if state[0] == 0 or self.result(state,'LEFT') in self.obstacles:
            possible_actions.remove('LEFT')
        if state[0] == (self.width-1)  or self.result(state,'RIGHT') in self.obstacles:
            possible_actions.remove('RIGHT')
        return possible_actions

    def result(self, state, action):
        new_state = list(state)
        if action == 'UP':
            new_state[1] -= 1
        if action == 'DOWN':
            new_state[1] += 1
        if action == 'LEFT':
            new_state[0] -= 1
        if action == 'RIGHT':
            new_state[0] += 1
        return tuple(new_state)

    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """
        return state == self.goal
    
    def h(self,node):
        current_state = list(node.state)
        goal_state = list(self.goal)
        return math.sqrt( (goal_state[0]-current_state[0])**2 + (goal_state[1]-current_state[1])**2)
    
    def h1(self,node):
        current_state = list(node.state)
        goal_state = list(self.goal)
        return(abs((goal_state[0]-current_state[0])+(goal_state[1]-current_state[1])))
    
new_maze = maze_problem(20,20,True)

,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,


# Question 2H (Advanced) 1 point 

Implement finding a path through generated mazes using Search and 
visualize the resulting solution using ipythonblocks. 

Do a comparison in terms of measured with tracing running time and 
space complexity of BFGS, DGFS, Greeedy GS, A* GS for 100 randomly generated mazes 
of 20 by 20 grids using two heuristics (Manhattan and Straight-Line distance). 

Show two tables (one for space and one for time complexity) with the results 
of 6 configurations (BFGS, DGFS, Greedy with Manhattan, Greedy with Straight-line, A* with Manhattan, A* with Straight-line). Which heuristics works better for the maze solving problem? 



In [16]:
# YOUR CODE GOES HERE 
# implement searching and visualize solution with blocks
# will need to keep obstacles in visualization
# compare BFGS,DFGS,Greedy,A* for 100 20x20 grids using each of manhattan and straight line distance
# show 2 tables, one for space and one for time complexity for each of the search algs and heuristics

import numpy as np

def visualize_solution(problem,node):
    
    solution = node.solution()
    temp_node = problem.initial
    for action in solution:
        problem.grid[temp_node[1],temp_node[0]] = (0,255,0)
        temp_node = problem.result(temp_node,action)
    problem.grid.show()

    
space_complexity_results = np.zeros(6)
time_complexity_results = np.zeros(6)
count = 0

while count < 100:
    new_maze = maze_problem(20,20)
    (node,time_complexity,space_complexity) = breadth_first_graph_search_complexity(new_maze)
    if node is None:
        continue
    time_complexity_results[0] += time_complexity
    space_complexity_results[0] += space_complexity
                           
    (node,time_complexity,space_complexity) = depth_first_graph_search_complexity(new_maze)
    time_complexity_results[1] += time_complexity
    space_complexity_results[1] += space_complexity
                           
    (node,time_complexity,space_complexity) = greedy_best_first_search_complexity(new_maze)
    time_complexity_results[2] += time_complexity
    space_complexity_results[2] += space_complexity
    (node,time_complexity,space_complexity) = greedy_best_first_search_complexity(new_maze,new_maze.h1)
    time_complexity_results[3] += time_complexity
    space_complexity_results[3] += space_complexity
                           
    (node,time_complexity,space_complexity) = astar_search_graph_complexity(new_maze)
    time_complexity_results[4] += time_complexity
    space_complexity_results[4] += space_complexity
    (node,time_complexity,space_complexity) = astar_search_graph_complexity(new_maze,new_maze.h1)
    time_complexity_results[5] += time_complexity
    space_complexity_results[5] += space_complexity
                           
    count += 1

time_complexity_results = time_complexity_results[:] / count
space_complexity_results = space_complexity_results[:] / count

if node is not None:
    visualize_solution(new_maze,node)

alg_labels = ["BFGS","DFGS","Greedy GS - linear","Greedy GS - manhattan","A* GS - linear","A* GS - manhattan"]
print("%-30s\t%-20s\t%-20s" % ("Algorithm","Time Complexity","Space Complexity"))
print('---------------------------------------------------------------------------')
for i in range(6):
    print("%-30s\t%-20.2f\t%-20.2f" % (alg_labels[i],time_complexity_results[i],space_complexity_results[i]))

,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,


Algorithm                     	Time Complexity     	Space Complexity    
---------------------------------------------------------------------------
BFGS                          	106.79              	8.43                
DFGS                          	108.32              	9.86                
Greedy GS - linear            	45.67               	10.05               
Greedy GS - manhattan         	57.98               	9.28                
A* GS - linear                	71.56               	8.97                
A* GS - manhattan             	74.39               	9.06                


**Question: Which heuristics works better for the maze solving problem?**  
**Ans:** Based on the results, *Greedy* graph search appears to work best out of all the algorithms as, although most of the algorithms have similar space complexities, the time complexity of Greedy is lowest, regardless of the heuristic used.  
In both the Greedy and A* searches, the *linear* heuristic appeared to be the best for the application as it resulted in a lower time complexity, with similar space complexities to the manhattan heuristic.